<a href="https://colab.research.google.com/github/VMBoehm/SDSS_PAE/blob/main/LSTM_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fitting the Density Estimator

In [1]:
## todo add forked repo as origin or upstream
## check paths
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#! pip install --user -e /global/u2/v/vboehm/codes/SIG_GIS/

In [3]:
import sys

In [4]:
sys.path.append('/global/u2/v/vboehm/codes/SIG_GIS/')

In [5]:
from sig_gis import *

In [6]:
from sig_gis.GIS import *

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [18]:
### settings
# user defined span (following Yip et al and Portillo et al)
root_models     = '/global/cscratch1/sd/vboehm/Models/SDSS_AE/'

root_encoded    = '/global/cscratch1/sd/vboehm/Datasets/encoded/sdss/'
root_decoded    = '/global/cscratch1/sd/vboehm/Datasets/decoded/sdss/'
root_model_data = '/global/cscratch1/sd/vboehm/Datasets/sdss/by_model/'
root_data       = '/global/cscratch1/sd/vboehm/Datasets'

root_prepped    = os.path.join(root_data,'sdss/prepped')

label           = 'galaxies_quasars_bins1000_wl3388-8318'
label_          = label+'_minz005_maxz036_minSN50'
label_2         = label+'_minz01_maxz036_minSN50_good'+'_10_fully_connected_mean_div'

seed            = 8720

latent_dim      = 10

wlmin, wlmax    = (3388,8318)
fixed_num_bins  = 1000
wl_range      = (np.log10(wlmin),np.log10(wlmax))
# new binning 
new_wl        = np.logspace(wl_range[0],wl_range[1],fixed_num_bins+1)


alpha         = (0.8,0.8)

maxiter       = 2000

label_3       = label_2+"alpha%d_maxiter%d"%(int(alpha[0]*10),maxiter)

In [10]:
encoded_train, encoded_valid, encoded_test = np.load(os.path.join(root_encoded,'encoded_%s.npy'%label_2), allow_pickle=True)
decoded_train, decoded_valid, decoded_test, mean = np.load(os.path.join(root_decoded,'decoded_%s.npy'%label_2), allow_pickle=True)

In [11]:
seed = 8365

In [12]:
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    device = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = torch.device('cpu')

In [13]:
encoded_train = torch.tensor(encoded_train).to(device)
encoded_valid = torch.tensor(encoded_valid).to(device)

In [14]:
try:
    model =  torch.load('GIS_%s_seed%d'%(label_3,seed))
except:
    model = train_GIS(encoded_train, data_validate=encoded_valid, alpha=torch.tensor(alpha).to(device), max_iter=maxiter)

Fit A: Time: 2.15651953125 Wasserstein Distance: [45.91404724121094, 12.394314765930176, 5.571999549865723, 3.9029715061187744, 2.3931171894073486, 1.9728652238845825, 1.7180386781692505, 1.100205898284912]
Fit spline Time: 0.129157470703125
logp: -786.1258544921875 -781.6680908203125 time: 3.782163381576538 iteration: 1 best: 1
Fit A: Time: 0.9689412841796875 Wasserstein Distance: [36.65995407104492, 9.188272476196289, 3.2290711402893066, 2.666271924972534, 2.6020290851593018, 2.3351023197174072, 1.7353558540344238, 0.878600537776947]
Fit spline Time: 0.11419305419921875
logp: -517.3600463867188 -514.3363647460938 time: 1.1227669715881348 iteration: 2 best: 2
Fit A: Time: 0.9640181274414062 Wasserstein Distance: [29.705848693847656, 6.537685394287109, 6.335050106048584, 2.306593418121338, 1.5631310939788818, 1.445253610610962, 1.4276739358901978, 1.049826741218567]
Fit spline Time: 0.09611196899414062
logp: -342.6133117675781 -340.5747375488281 time: 1.0948331356048584 iteration: 3 be

In [19]:
torch.save(model, os.path.join(root_models,'GIS_%s_%d'%(label_3,seed)))

In [ ]:
#torch.save(model, 'GIS_%s_seed%d'%(label_2,seed))
model.evaluate_density(encoded_valid[0:100]).cpu()